In [1]:
import pandas as pd
from tqdm import tqdm
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from pywinauto.keyboard import send_keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [6]:
driver = webdriver.Chrome()
driver.get("https://inmuebles.mercadolibre.com.ar/departamentos/alquiler/capital-federal/departamento-alquiler_NoIndex_True#unapplied_filter_id%3DROOMS%26unapplied_filter_name%3DAmbientes%26unapplied_value_id%3D%5B2-2%5D%26unapplied_value_name%3D2+ambientes%26unapplied_autoselect%3Dfalse")

In [ ]:

variables = [
"id", "tipo", "titulo", "direccion",
"m2_totales", "m2_cubiertos", "m2_balcon",
"ambientes", "dormitorios", "baños", "cocheras",
"admite_mascotas", "piso", "antiguedad",
"disposicion", "orientacion", "amoblado",
"moneda_precio", "precio", "expensas"
]

# Inicializar las listas para almacenar los datos
data = {variable: [] for variable in variables}

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//body")))

#---------------------------------------------------------------------------------------------------------------------------
# Inicio del scrapping

# Generación de las 48 publicaciones por página
XPATH_POST = "//section[@class='ui-search-results']//ol//div//li[@class='ui-search-layout__item']//div[@class='ui-search-result__wrapper']"

posts = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_POST)))

for i, publicacion in enumerate(tqdm(posts, desc="Procesando publicaciones", total=len(posts))):

    try:
        # Abrir cada una de las publicaciones
        publicacion.click()

        # Capturar todas las ventanas y seleccionar la última
        pestañas = driver.window_handles
        driver.switch_to.window(pestañas[-1])

        # Esperar a que la página cargue completamente
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//body")))

        # Lista de las rutas web de interés:
        XPATH_ID               = "//span[@class='ui-pdp-color--BLACK ui-pdp-family--SEMIBOLD']"
        XPATH_TIPO             = "//span[@class='ui-pdp-subtitle']"
        XPATH_TITULO           = "//h1[@class='ui-pdp-title']"
        XPATH_DIRECCION        = "//div[@class='ui-vip-location']//div[@class='ui-pdp-media ui-vip-location__subtitle ui-pdp-color--BLACK']//div[@class='ui-pdp-media__body']//p[@class='ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--REGULAR ui-pdp-media__title']"
        XPATH_MONEDA           = "//span[@class='andes-money-amount__currency-symbol']"
        XPATH_PRECIO           = "//span[@class='andes-money-amount__fraction']"
        XPATH_EXPENSAS         = "//p[@class='ui-pdp-color--GRAY ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-maintenance-fee-ltr']"
        XPATH_VER_TABLA        = "//span[@title='Ver todas las características']"
        XPATH_TABLA_TH         = "//table[@class='andes-table']//tbody[@class='andes-table__body']//tr[@class='andes-table__row ui-vpp-striped-specs__row']//div[@class='andes-table__header__container']"
        XPATH_TABLA_TD         = "//table[@class='andes-table']//tbody[@class='andes-table__body']//td[@class='andes-table__column andes-table__column--left andes-table__column--vertical-align-center ui-vpp-striped-specs__row__column']//span[@class='andes-table__column--value']"

        # Primera parte de los datos
        try:
            id = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_ID))).text
        except TimeoutError:
            id = "Sin dato"

        try:
            tipo = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_TIPO))).text
        except TimeoutError:
            tipo = "Sin dato"    

        try:
            titulo = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_TITULO))).text
        except TimeoutError:
            titulo = "Sin dato" 

        try:
            direccion = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_DIRECCION))).text
        except TimeoutError:
            direccion = "Sin dato" 

        try:
            moneda_precio = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_MONEDA))).text
        except TimeoutError:
            moneda_precio = "Sin dato" 

        try:
            precio = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, XPATH_PRECIO))).text
        except TimeoutError:
            precio = "Sin dato" 
        


        try:
            ver_mas_boton = WebDriverWait(driver, 110).until(EC.presence_of_element_located((By.XPATH, XPATH_VER_TABLA)))

            ver_mas_boton.click()

            # Esperar a que la página cargue completamente
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//body")))

            tabla_th    = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TH)))

            # Declarando variables de interés
            m2_totales      = "Sin dato"
            m2_cubiertos    = "Sin dato"
            m2_balcon       = "Sin dato"
            ambientes       = "Sin dato"
            dormitorios     = "Sin dato"
            baños           = "Sin dato"
            cocheras        = "Sin dato"
            admite_mascotas = "Sin dato"
            piso            = "Sin dato"
            antiguedad      = "Sin dato"
            disposicion     = "Sin dato"
            orientacion     = "Sin dato"
            amoblado        = "Sin dato"
            expensas        = "Sin dato"


            for i, categoria in enumerate(tabla_th):

                categoria = categoria.text
                
                if categoria == "Superficie total":

                    m2_totales = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Superficie cubierta":

                    m2_cubiertos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text
                
                elif categoria == "Superficie de balcón":

                    m2_balcon = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Ambientes":

                    ambientes = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Dormitorios":

                    dormitorios = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Baños":

                    baños = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Cocheras":

                    cocheras = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Admite mascotas":

                    admite_mascotas = WebDriverWait(driver, 2).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Número de piso de la unidad":

                    piso = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Antiguedad" or categoria == "Antigüedad":

                    antiguedad = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Disposición":

                    disposicion = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Orientación":

                    orientacion = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text


                elif categoria == "Amoblado":

                    amoblado = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

                elif categoria == "Expensas":

                    expensas = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, XPATH_TABLA_TD)))[i].text

        except TimeoutError:
            print("Algo falló en la segunda tanda de información")



        # Añadir los datos a las listas
        data["id"].append(id)
        data["tipo"].append(tipo)
        data["titulo"].append(titulo)
        data["direccion"].append(direccion)
        data["m2_totales"].append(m2_totales)
        data["m2_cubiertos"].append(m2_cubiertos)
        data["m2_balcon"].append(m2_balcon)
        data["ambientes"].append(ambientes)
        data["dormitorios"].append(dormitorios)
        data["baños"].append(baños)
        data["cocheras"].append(cocheras)
        data["admite_mascotas"].append(admite_mascotas)
        data["piso"].append(piso)
        data["antiguedad"].append(antiguedad)
        data["disposicion"].append(disposicion)
        data["orientacion"].append(orientacion)
        data["amoblado"].append(amoblado)
        data["moneda_precio"].append(moneda_precio)
        data["precio"].append(precio)
        data["expensas"].append(expensas)

    except TimeoutError:
        print(f"Hubo conflictos al abrir la publicación N° {i+1}")
    finally:
        driver.switch_to.window(pestañas[0])


df = pd.DataFrame(data)
df.to_csv(f"df_.csv", encoding="utf-8")

In [14]:
# Leemos todos los dfs generados
df_concat = []

for i in range(1, 42):
    filename = f"./dfs/df_{i}.csv"
    df       = pd.read_csv(filename)
    df_concat.append(df)

In [24]:
# Concatenar los distintos dfs
df_alquileres = pd.concat(df_concat, ignore_index=False)

In [27]:
# Eliminando la columna Unnamed: 0
df_alquileres = df_alquileres.drop("Unnamed: 0", axis=1)

In [28]:
# Exportando a formato csv el dataframe final
df_alquileres.to_csv("./dfs/df_alquileres.csv", index=False)

In [31]:
df_alquileres.head()

,id,tipo,titulo,direccion,m2_totales,m2_cubiertos,m2_balcon,ambientes,dormitorios,baños,cocheras,admite_mascotas,piso,antiguedad,disposicion,orientacion,amoblado,moneda_precio,precio,expensas
0,#1406312133,Departamento en Alquiler,Departamento Alquiler Saavedra Monoambiente Co...,"Holmberg Al 3700, Saavedra, Capital Federal, C...",35 m²,27 m²,Sin dato,1,0,1,1,Sin dato,Sin dato,0 años,Contrafrente,Oeste,No,U$S,400,0 ARS
1,#1652330026,Departamento en Alquiler,Alquiler De Departamento 3 Ambientes En Recoleta,"Montevideo Al 1200. Entre Arenales Y Juncal, R...",114 m²,105 m²,9 m²,3,2,1,0,No,5,50 años,Frente,Este,No,U$S,1.200,90000 ARS
2,#1406090353,Departamento en Alquiler,Alquiler De Departamento 3 Ambientes Con Coche...,"Aguirre Al 300, Villa Crespo, Capital Federal,...",65 m²,58 m²,Sin dato,3,2,2,0,Sin dato,Sin dato,15 años,Contrafrente,Sin dato,No,U$S,650,80000 ARS
3,#1656558384,Departamento en Alquiler,"Sls Lux Tower, 2 Ambientes - Departamento En V...","Rosario Vera Peñaloza Al 300, Puerto Madero, C...",120 m²,120 m²,Sin dato,2,1,1,2,Sin dato,Sin dato,2 años,Frente,Norte,No,U$S,3.500,1 ARS
4,#1649136888,Departamento en Alquiler,Departamento En Palermo Chico,"Figueroa Alcorta 3400, Palermo, Capital Federa...",120 m²,110 m²,Sin dato,3,2,2,1,Sin dato,Sin dato,0 años,Sin dato,Sin dato,No,U$S,1.800,1 ARS
